In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

In [ ]:
# Set paths (adjust if using a custom dataset)
train_dir = 'data/train'  # Should have 'dogs' and 'cats' subfolders
val_dir = 'data/validation'


In [ ]:
# Image preprocessing
img_size = 150
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, shear_range=0.2,
                                   zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)